# Лабораторная работа №8 Выявление психологических отклонений по тексту
#### Предмет: Методы интеллектуального анализа текстов
#### Студент: Леонова Алина, СБ № 1032212306, НФИмд-01-21

## 8.4. Обучить fasttext-классификатор, сравнить качество классификации с предобученными эмбеддингами и обученными с нуля при классификации. 

**Задача**: используя известные инструменты анализа текста, обучить бинарный классификатор, предсказывающий наличие у автора эссе депрессии.

isanlp - для готовых аннотаций; catboost - для фильтрации признаков (включите поддержку gpu, если вы в среде colab)

In [1]:
! pip install -Uq git+https://github.com/IINemo/isanlp.git catboost

     |████████████████████████████████| 76.8 MB 1.1 MB/s 
     |████████████████████████████████| 1.0 MB 54.2 MB/s 


Данные для обучения моделей:

In [2]:
# добавляю архив в файлы ручками
! tar -xzvf PsyHack_RUDN_train.tar.gz

PsyHack.RUDN2019_train/PsyHack_RUDN_train.csv
PsyHack.RUDN2019_train/PsyHack_RUDN_train_annots.pkl
PsyHack.RUDN2019_train/quickstart.py
PsyHack.RUDN2019_train/description.txt


Данные для тестирования моделей: 

In [3]:
# добавляю архив в файлы ручками
! tar -xzvf PsyHack.RUDN2019_test.tar.gz

PsyHack.RUDN2019_test/PsyHack_RUDN_test.csv
PsyHack.RUDN2019_test/PsyHack_RUDN_test_annots.pkl


### Проделывание всего с данными для обучения, разделенными на train и test

In [4]:
import pandas as pd
import numpy as np
import pickle

data = pd.read_csv('PsyHack.RUDN2019_train/PsyHack_RUDN_train.csv', sep='\t')  # тексты
data['annots'] = pickle.load(open('PsyHack.RUDN2019_train/PsyHack_RUDN_train_annots.pkl', 'rb'))  # результаты автоматической аннотации
data.head(10).tail(5)

,ID,label,text,annots
5,128,Non-depression,К каждому человеку я отношусь изначально позит...,{'text': 'К каждому человеку я отношусь изнача...
6,157,Depression,"«Я, другие, мир».Все меняется в этом мире. Мен...","{'text': '«Я, другие, мир».Все меняется в этом..."
7,294,Depression,"Приветсвую, меня зовут Никита. Мне 20 полных л...","{'text': 'Приветсвую, меня зовут Никита. Мне 2..."
8,67,Non-depression,В данном эссе я попытаюсь дать оценку своему х...,{'text': 'В данном эссе я попытаюсь дать оценк...
9,117,Non-depression,"Как много всего нас окружает, и как мало мы за...","{'text': 'Как много всего нас окружает, и как ..."


In [5]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(data, test_size=0.1, random_state=42)

In [6]:
train

,ID,label,text,annots
25,113,Non-depression,"Я, другие и мир - неотделимые друг от друга ве...","{'text': 'Я, другие и мир - неотделимые друг о..."
127,275,Depression,Я по своим взглядам на жизнь тяготею к буддийс...,{'text': 'Я по своим взглядам на жизнь тяготею...
201,135,Non-depression,"Я, другие, мир. Я живу в самой большой по площ...","{'text': 'Я, другие, мир. Я живу в самой больш..."
97,300,Non-depression,"У меня много друзей, близких и далёких. По мно...","{'text': 'У меня много друзей, близких и далёк..."
16,282,Depression,"Я, другие, Мир. Человек - это звучит гордо. М....","{'text': 'Я, другие, Мир. Человек - это звучит..."
...,...,...,...,...
106,14,Depression,Я Сигачева Людмила Алексеевна. Родилась 12 июл...,{'text': 'Я Сигачева Людмила Алексеевна. Родил...
14,269,Non-depression,"Начну с того, что с родственниками у меня не с...","{'text': 'Начну с того, что с родственниками у..."
92,201,Non-depression,Общение играет важную роль в жизни каждого чел...,{'text': 'Общение играет важную роль в жизни к...
179,19,Non-depression,"Я,другие,мир.Что для меня это значит? Под этой...","{'text': 'Я,другие,мир.Что для меня это значит..."


In [7]:
test

,ID,label,text,annots
132,93,Non-depression,"Я довольна своей жизнью. Конечно, есть что-то,...","{'text': 'Я довольна своей жизнью. Конечно, ес..."
148,256,Non-depression,"Я думаю, что люди, которые окружают меня кажды...","{'text': 'Я думаю, что люди, которые окружают ..."
93,144,Non-depression,В жизни каждого человека очень важную роль игр...,{'text': 'В жизни каждого человека очень важну...
180,314,Non-depression,"Я, другие и мир. Полагаю, что сперва стоит опр...","{'text': 'Я, другие и мир. Полагаю, что сперва..."
15,53,Non-depression,Человек с момента своего рождения всегда наход...,{'text': 'Человек с момента своего рождения вс...
115,254,Depression,У меня диагностировано БАР I типа. Сейчас я на...,{'text': 'У меня диагностировано БАР I типа. С...
172,212,Non-depression,"Я довольно-таки общительный человек, поэтому, ...","{'text': 'Я довольно-таки общительный человек,..."
209,57,Non-depression,"""Обо мне и отношениях с другими людьми"" Достат...","{'text': '""Обо мне и отношениях с другими людь..."
75,192,Depression,"Если бы можно было ставить статусы отношений, ...",{'text': 'Если бы можно было ставить статусы о...
142,233,Depression,"Я, похоже, все-таки счастливый человек. Живу н...","{'text': 'Я, похоже, все-таки счастливый челов..."


#### Предобработка данных

In [8]:
# Очистка данных
import nltk
import string, re
from collections import Counter, OrderedDict
from nltk.tokenize import RegexpTokenizer

tokenizer = RegexpTokenizer('\W+', gaps=True)
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('russian')

# минимальная обработка текста
def clean(text):
    text = re.sub('ё','е',str(text))
    text = re.sub('\\n',' ',str(text)) # удаляет переносы строк
    text = re.sub('\s{2,}', " ", text) # удаляет лишние пробелы
    return text

# чистит от стоп слов токенезированный текст
def clean_from_stopwords(text):
    text = [token for token in text if re.match(r'[а-яА-Я]', token) and token not in stopwords]
    #text = re.sub('ё','е',str(text))
    text = re.sub(r'[^\яа-яА-Я0-9@]',' ',str(text))
    text = re.sub(r'\w*\d+\w*', '', text) # удаляет цифры
    text = re.sub('\s{2,}', " ", text) # удаляет лишние пробелы
    return text

train['text'] = train['text'].apply(clean)
test['text'] = test['text'].apply(clean)

# токенизирование
train['text_tokenized'] = train.text.map(lambda row: tokenizer.tokenize(row.lower()))
test['text_tokenized'] = test.text.map(lambda row: tokenizer.tokenize(row.lower()))

train['cleaned_text'] = train['text_tokenized'].apply(clean_from_stopwords)
test['cleaned_text'] = test['text_tokenized'].apply(clean_from_stopwords)

train

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


,ID,label,text,annots,text_tokenized,cleaned_text
25,113,Non-depression,"Я, другие и мир - неотделимые друг от друга ве...","{'text': 'Я, другие и мир - неотделимые друг о...","[я, другие, и, мир, неотделимые, друг, от, дру...",другие мир неотделимые друг друга вещи самое ...
127,275,Depression,Я по своим взглядам на жизнь тяготею к буддийс...,{'text': 'Я по своим взглядам на жизнь тяготею...,"[я, по, своим, взглядам, на, жизнь, тяготею, к...",своим взглядам жизнь тяготею буддийской филос...
201,135,Non-depression,"Я, другие, мир. Я живу в самой большой по площ...","{'text': 'Я, другие, мир. Я живу в самой больш...","[я, другие, мир, я, живу, в, самой, большой, п...",другие мир живу самой большой площади стране ...
97,300,Non-depression,"У меня много друзей, близких и далеких. По мно...","{'text': 'У меня много друзей, близких и далёк...","[у, меня, много, друзей, близких, и, далеких, ...",друзей близких далеких многим вопросам сходим...
16,282,Depression,"Я, другие, Мир. Человек - это звучит гордо. М....","{'text': 'Я, другие, Мир. Человек - это звучит...","[я, другие, мир, человек, это, звучит, гордо, ...",другие мир человек это звучит гордо м горький...
...,...,...,...,...,...,...
106,14,Depression,Я Сигачева Людмила Алексеевна. Родилась 12 июл...,{'text': 'Я Сигачева Людмила Алексеевна. Родил...,"[я, сигачева, людмила, алексеевна, родилась, 1...",сигачева людмила алексеевна родилась июля год...
14,269,Non-depression,"Начну с того, что с родственниками у меня не с...","{'text': 'Начну с того, что с родственниками у...","[начну, с, того, что, с, родственниками, у, ме...",начну родственниками гладко свои разногласия ...
92,201,Non-depression,Общение играет важную роль в жизни каждого чел...,{'text': 'Общение играет важную роль в жизни к...,"[общение, играет, важную, роль, в, жизни, кажд...",общение играет важную роль жизни каждого чело...
179,19,Non-depression,"Я,другие,мир.Что для меня это значит? Под этой...","{'text': 'Я,другие,мир.Что для меня это значит...","[я, другие, мир, что, для, меня, это, значит, ...",другие мир это значит фразой сразу голове всп...


In [9]:
# Добавление лемматизированного текста
import itertools

symbol_mapper = {
    'ё': 'е',
    '“': '«',
    '”': '»',
    '—': '-',
    '–': '-'
}

def pos_lemmas(annot):
    """ Returns: Мама_NOUN мыть_VERB рама_NOUN """
    
    postags = itertools.chain.from_iterable(annot['postag'])
    lemmas = itertools.chain.from_iterable(annot['lemma'])
    result = [(next(lemmas), next(postags)) for _ in annot['tokens']]
    result = ' '.join(['_'.join(pair) for pair in result if not pair[0].isdigit()])  # remove numbers
    
    for symbol in symbol_mapper.keys():
        result = result.replace(symbol, symbol_mapper[symbol])
    
    return result


train['text_lemmatized'] = train.annots.map(pos_lemmas)
test['text_lemmatized'] = train.annots.map(pos_lemmas)

train

,ID,label,text,annots,text_tokenized,cleaned_text,text_lemmatized
25,113,Non-depression,"Я, другие и мир - неотделимые друг от друга ве...","{'text': 'Я, другие и мир - неотделимые друг о...","[я, другие, и, мир, неотделимые, друг, от, дру...",другие мир неотделимые друг друга вещи самое ...,"я_PRON ,_PUNCT другой_ADJ и_CCONJ мир_NOUN -_P..."
127,275,Depression,Я по своим взглядам на жизнь тяготею к буддийс...,{'text': 'Я по своим взглядам на жизнь тяготею...,"[я, по, своим, взглядам, на, жизнь, тяготею, к...",своим взглядам жизнь тяготею буддийской филос...,я_PRON по_ADP свой_DET взгляд_NOUN на_ADP жизн...
201,135,Non-depression,"Я, другие, мир. Я живу в самой большой по площ...","{'text': 'Я, другие, мир. Я живу в самой больш...","[я, другие, мир, я, живу, в, самой, большой, п...",другие мир живу самой большой площади стране ...,"я_PRON ,_PUNCT другой_ADJ ,_PUNCT мир_NOUN ._P..."
97,300,Non-depression,"У меня много друзей, близких и далеких. По мно...","{'text': 'У меня много друзей, близких и далёк...","[у, меня, много, друзей, близких, и, далеких, ...",друзей близких далеких многим вопросам сходим...,"у_ADP я_PRON много_ADV друг_NOUN ,_PUNCT близк..."
16,282,Depression,"Я, другие, Мир. Человек - это звучит гордо. М....","{'text': 'Я, другие, Мир. Человек - это звучит...","[я, другие, мир, человек, это, звучит, гордо, ...",другие мир человек это звучит гордо м горький...,"я_PRON ,_PUNCT другой_ADJ ,_PUNCT мир_PROPN ._..."
...,...,...,...,...,...,...,...
106,14,Depression,Я Сигачева Людмила Алексеевна. Родилась 12 июл...,{'text': 'Я Сигачева Людмила Алексеевна. Родил...,"[я, сигачева, людмила, алексеевна, родилась, 1...",сигачева людмила алексеевна родилась июля год...,я_PRON Сигачева_PROPN Людмила_PROPN Алексеевна...
14,269,Non-depression,"Начну с того, что с родственниками у меня не с...","{'text': 'Начну с того, что с родственниками у...","[начну, с, того, что, с, родственниками, у, ме...",начну родственниками гладко свои разногласия ...,"начать_VERB с_ADP то_PRON ,_PUNCT что_SCONJ с_..."
92,201,Non-depression,Общение играет важную роль в жизни каждого чел...,{'text': 'Общение играет важную роль в жизни к...,"[общение, играет, важную, роль, в, жизни, кажд...",общение играет важную роль жизни каждого чело...,общение_NOUN играть_VERB важный_ADJ роль_NOUN ...
179,19,Non-depression,"Я,другие,мир.Что для меня это значит? Под этой...","{'text': 'Я,другие,мир.Что для меня это значит...","[я, другие, мир, что, для, меня, это, значит, ...",другие мир это значит фразой сразу голове всп...,"я_PRON ,_PUNCT другой_ADJ ,_PUNCT мир_NOUN ._P..."


In [10]:
test

,ID,label,text,annots,text_tokenized,cleaned_text,text_lemmatized
132,93,Non-depression,"Я довольна своей жизнью. Конечно, есть что-то,...","{'text': 'Я довольна своей жизнью. Конечно, ес...","[я, довольна, своей, жизнью, конечно, есть, чт...",довольна своей жизнью хотела улучшить это воо...,NaN
148,256,Non-depression,"Я думаю, что люди, которые окружают меня кажды...","{'text': 'Я думаю, что люди, которые окружают ...","[я, думаю, что, люди, которые, окружают, меня,...",думаю люди которые окружают каждый день своем...,NaN
93,144,Non-depression,В жизни каждого человека очень важную роль игр...,{'text': 'В жизни каждого человека очень важну...,"[в, жизни, каждого, человека, очень, важную, р...",жизни каждого человека очень важную роль игра...,NaN
180,314,Non-depression,"Я, другие и мир. Полагаю, что сперва стоит опр...","{'text': 'Я, другие и мир. Полагаю, что сперва...","[я, другие, и, мир, полагаю, что, сперва, стои...",другие мир полагаю сперва стоит определить ка...,NaN
15,53,Non-depression,Человек с момента своего рождения всегда наход...,{'text': 'Человек с момента своего рождения вс...,"[человек, с, момента, своего, рождения, всегда...",человек момента своего рождения находится окр...,NaN
115,254,Depression,У меня диагностировано БАР I типа. Сейчас я на...,{'text': 'У меня диагностировано БАР I типа. С...,"[у, меня, диагностировано, бар, i, типа, сейча...",диагностировано бар типа нахожусь клинике нев...,NaN
172,212,Non-depression,"Я довольно-таки общительный человек, поэтому, ...","{'text': 'Я довольно-таки общительный человек,...","[я, довольно, таки, общительный, человек, поэт...",довольно таки общительный человек поэтому исх...,NaN
209,57,Non-depression,"""Обо мне и отношениях с другими людьми"" Достат...","{'text': '""Обо мне и отношениях с другими людь...","[обо, мне, и, отношениях, с, другими, людьми, ...",обо отношениях другими людьми достаточно инте...,NaN
75,192,Depression,"Если бы можно было ставить статусы отношений, ...",{'text': 'Если бы можно было ставить статусы о...,"[если, бы, можно, было, ставить, статусы, отно...",ставить статусы отношений это точно сложно та...,NaN
142,233,Depression,"Я, похоже, все-таки счастливый человек. Живу н...","{'text': 'Я, похоже, все-таки счастливый челов...","[я, похоже, все, таки, счастливый, человек, жи...",похоже таки счастливый человек живу свете лет...,NaN


In [11]:
# Создание текстовых файлов для обучения fasttext модели с лейблом и текстом
def make_txt(data, column, file_name):
    with open(file_name, 'w') as f:
        for each_text, each_label in zip(data[column], data['label']):
            f.writelines(f'__label__{each_label} {each_text}\n')

# файлы с минимально обработаннымм текстами
make_txt(train, 'text', 'train-text.txt')
make_txt(test, 'text', 'test-text.txt')

print('Данные для обучения:')
! head -n 10 train-text.txt

Данные для обучения:
__label__Non-depression Я, другие и мир - неотделимые друг от друга вещи. Самое важное для меня среди данных слов - "я". Мой внутренний мир, мой комфорт, мое счастье, здоровье для меня на первом месте. На мой взгляд, так должно быть у каждого человека. Ведь только обретя гармонию внутри себя, можно устроить свою жизнь наилучшим образом. От хорошего эмоционального и физического самочувствия зависит продуктивность каждого дня, отношения с окружающими людьми. Как мне кажется то, что мы называем удачей есть ничто иное, как та положительная энергия, которая формируется внутри нас самих и освещает все вокруг, придавая каждому событию или действию положительную окраску. Так мы встречаем нужных людей, совершаем хорошие сделки, качественно выполняем работу и исполняем заветные мечты. Но, к сожалению, иногда происходят события, не зависящие от нас. Здесь стоит упомянуть "других". Люди бывают разные, и нам не дано знать, что происходит у них в голове. В повседневной жизни мы 

In [12]:
# файлы с текстами без знаков препинания и стоп слов
make_txt(train, 'cleaned_text', 'train-cleaned.txt')
make_txt(test, 'cleaned_text', 'test-cleaned.txt')

print('Данные для обучения:')
! head -n 10 train-cleaned.txt

Данные для обучения:
__label__Non-depression  другие мир неотделимые друг друга вещи самое важное среди данных слов внутренний мир комфорт мое счастье здоровье первом месте взгляд должно каждого человека обретя гармонию внутри устроить жизнь наилучшим образом хорошего эмоционального физического самочувствия зависит продуктивность каждого дня отношения окружающими людьми кажется называем удачей ничто иное та положительная энергия которая формируется внутри самих освещает вокруг придавая каждому событию действию положительную окраску встречаем нужных людей совершаем хорошие сделки качественно выполняем работу исполняем заветные мечты сожалению происходят события зависящие стоит упомянуть других люди бывают разные нам дано знать происходит голове повседневной жизни часто можем столкнуться грубостью невежеством вынуждены тратить свое время неприятных нам людей бесконечно говорить просто стоит принимать негатив некоторых ситуациях это невозможно взгляд таким человеком которого злит раздража

In [13]:
! pip install fasttext

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 68 kB 2.9 MB/s 
  Using cached pybind11-2.10.0-py3-none-any.whl (213 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp37-cp37m-linux_x86_64.whl size=3168443 sha256=ed914946302085c4ac16cbcf85948c8adc511004b1d086bf2a5c219411a7b6ac
  Stored in directory: /root/.cache/pip/wheels/4e/ca/bf/b020d2be95f7641801a6597a29c8f4f19e38f9c02a345bab9b
Successfully built fasttext


In [14]:
import fasttext
from sklearn.metrics import f1_score

# Печать результатов обучения модели
def print_results(sample_size, precision, recall):
    precision = round(precision, 2)
    recall = round(recall, 2)
    print('Размер выборки = ', sample_size)
    print('Precision = ', precision)
    print('Recall = ', recall)

# Вычисление предсказаний для всех текстов и сравнение их с реальными
def preds_f1(model, test_column):
    pred = []
    for line in test_column:
        pred.append(model.predict(line)[0][0])

    f1 = f1_score('__label__'+test.label, pred, average='macro')
    print('\nf1_score = ', f1)

In [15]:
# Модель с базовыми параметрами на минимально обработанных данных
model0 = fasttext.train_supervised('train-text.txt')

print_results(*model0.test('test-text.txt'))
preds_f1(model0, test['text'])

Размер выборки =  23
Precision =  0.83
Recall =  0.83

f1_score =  0.45238095238095233


In [16]:
# Модель с базовыми параметрами на данных без знаков препинания и стоп слов
model1 = fasttext.train_supervised('train-cleaned.txt')

print_results(*model1.test('test-cleaned.txt'))
preds_f1(model1, test['cleaned_text'])

Размер выборки =  23
Precision =  0.83
Recall =  0.83

f1_score =  0.45238095238095233


Разницы не замечено

#### Теперь для той же задачи использую предобученную модель

In [17]:
! pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.3 MB 4.3 MB/s 
     |████████████████████████████████| 163 kB 65.0 MB/s 
     |████████████████████████████████| 7.6 MB 45.3 MB/s 


In [18]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained("mrm8488/distilroberta-base-finetuned-suicide-depression")

#model = AutoModelForSequenceClassification.from_pretrained("mrm8488/distilroberta-base-finetuned-suicide-depression")
classifier = pipeline("text-classification", model="mrm8488/distilroberta-base-finetuned-suicide-depression")

Downloading:   0%|          | 0.00/333 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/798k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/740 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/329M [00:00<?, ?B/s]

In [19]:
true_labels = []
for i in test.label:
  true_labels.append(i)

In [20]:
def predict_dep(text, dep_score):
  split_regex = re.compile(r'[.|!|?|…]')
  print('dep_score = ', dep_score)

  res = []
  k = 0
  for i in text:
    print(k)
    k+=1
    count_s = 0   # количество предложений
    count_dep = 0   # количество депрессивных предложений
    sentences = filter(lambda t: t, [t.strip() for t in split_regex.split(i)])
    for sent in sentences:
      is_dep = classifier(sent)[0]['label']
      if is_dep == 'LABEL_0':  # или наоборот...
        count_dep += 1

      count_s += 1

    d = count_dep/count_s   # процент депрессивных предложений в тексте
    print(d)
    if d > dep_score:
      print(('Depression'))
      res.append('Depression')
    else:
      res.append('Non-depression')
      print('Non-depression')

  print('res  = ', res)
  #print('true = ', true_labels)
  #print('f1_score = ', f1_score(true_labels, res, average='macro'))

In [ ]:
%%time
predict_dep(test.text, 0.05)
predict_dep(test.text, 0.1)
predict_dep(test.text, 0.15)
predict_dep(test.text, 0.2)
predict_dep(test.text, 0.3)
predict_dep(test.text, 0.4)

dep_score =  0.05
res  =  ['Non-depression', 'Depression', 'Non-depression', 'Depression', 'Depression', 'Non-depression', 'Depression', 'Non-depression', 'Non-depression', 'Depression', 'Non-depression', 'Non-depression', 'Depression', 'Non-depression', 'Depression', 'Depression', 'Non-depression', 'Non-depression', 'Non-depression', 'Non-depression', 'Depression', 'Non-depression', 'Non-depression']
true =  ['Non-depression', 'Non-depression', 'Non-depression', 'Non-depression', 'Non-depression', 'Depression', 'Non-depression', 'Non-depression', 'Depression', 'Depression', 'Non-depression', 'Non-depression', 'Depression', 'Non-depression', 'Non-depression', 'Non-depression', 'Non-depression', 'Non-depression', 'Non-depression', 'Non-depression', 'Non-depression', 'Non-depression', 'Non-depression']
f1_score =  0.5174825174825175
dep_score =  0.1
res  =  ['Non-depression', 'Non-depression', 'Non-depression', 'Depression', 'Depression', 'Non-depression', 'Depression', 'Non-depression',

Всё плохо. И спорно какое значение процента депрессивных предложений говорит о депрессии. Возможно, если найдётся одно депрессивное предложение, на этом можно остановиться и считать всё за депрессию. Или лучше не делить текст на предложения, а рассматривать целиком, но увеличить размер pipeline мне не удалось.

### Проделывание всего с тестовыми данными

In [21]:
train = data
train.head(10).tail(5)

,ID,label,text,annots
5,128,Non-depression,К каждому человеку я отношусь изначально позит...,{'text': 'К каждому человеку я отношусь изнача...
6,157,Depression,"«Я, другие, мир».Все меняется в этом мире. Мен...","{'text': '«Я, другие, мир».Все меняется в этом..."
7,294,Depression,"Приветсвую, меня зовут Никита. Мне 20 полных л...","{'text': 'Приветсвую, меня зовут Никита. Мне 2..."
8,67,Non-depression,В данном эссе я попытаюсь дать оценку своему х...,{'text': 'В данном эссе я попытаюсь дать оценк...
9,117,Non-depression,"Как много всего нас окружает, и как мало мы за...","{'text': 'Как много всего нас окружает, и как ..."


In [22]:
test = pd.read_csv('PsyHack.RUDN2019_test/PsyHack_RUDN_test.csv', sep='\t')  # тексты
test['annots'] = pickle.load(open('PsyHack.RUDN2019_test/PsyHack_RUDN_test_annots.pkl', 'rb'))  # результаты автоматической аннотации
test.head(10).tail(5)

,ID,label,text,annots
5,4547,NaN,"Понимает ли ребёнок, когда рождается, что он с...","{'text': 'Понимает ли ребёнок, когда рождается..."
6,12863,NaN,"Я, другие, мир. Сложный и в то же время просто...","{'text': 'Я, другие, мир. Сложный и в то же вр..."
7,3053,NaN,"Мир для меня не является каким-то открытием, к...",{'text': 'Мир для меня не является каким-то от...
8,7817,NaN,"Я точно знаю, что Мир, не есть совокупность ск...","{'text': 'Я точно знаю, что Мир, не есть совок..."
9,14654,NaN,"Сейчас у меня есть круг друзей, порядка десяти...","{'text': 'Сейчас у меня есть круг друзей, поря..."


In [23]:
tokenizer = RegexpTokenizer('\W+', gaps=True)

# Предобработка данных
train['text'] = train['text'].apply(clean)
test['text'] = test['text'].apply(clean)

train['text_tokenized'] = train.text.map(lambda row: tokenizer.tokenize(row.lower()))
test['text_tokenized'] = test.text.map(lambda row: tokenizer.tokenize(row.lower()))

train['cleaned_text'] = train['text_tokenized'].apply(clean_from_stopwords)
test['cleaned_text'] = test['text_tokenized'].apply(clean_from_stopwords)

In [24]:
test

,ID,label,text,annots,text_tokenized,cleaned_text
0,19163,NaN,В плане отношений с другими люди делятся на дв...,{'text': 'В плане отношений с другими люди дел...,"[в, плане, отношений, с, другими, люди, делятс...",плане отношений другими люди делятся типа экс...
1,7028,NaN,"Я в этом мире как муравей в муравейнике, как л...",{'text': 'Я в этом мире как муравей в муравейн...,"[я, в, этом, мире, как, муравей, в, муравейник...",мире муравей муравейнике лиса погоне едой чув...
2,5070,NaN,"Современные люди стали жестоки,к нашему миру и...","{'text': 'Современные люди стали жестоки,к наш...","[современные, люди, стали, жестоки, к, нашему,...",современные люди стали жестоки нашему миру лю...
3,17580,NaN,Люди ко мне относится довольна странно я обычн...,{'text': 'Люди ко мне относится довольна стран...,"[люди, ко, мне, относится, довольна, странно, ...",люди ко относится довольна странно обычны чел...
4,17445,NaN,"Ну так как мне 18 лет, то я еще не совсем пони...","{'text': 'Ну так как мне 18 лет, то я еще не с...","[ну, так, как, мне, 18, лет, то, я, еще, не, с...",лет понимаю происходит вокруг кажется всемогу...
...,...,...,...,...,...,...
249,6928,NaN,"Мир-слово, имеющее несколько значений. Оно упо...","{'text': 'Мир-слово, имеющее несколько значени...","[мир, слово, имеющее, несколько, значений, оно...",мир слово имеющее несколько значений оно упот...
250,3540,NaN,Мир – интересная штука. Сейчас я смотрю на нег...,{'text': 'Мир – интересная штука. Сейчас я смо...,"[мир, интересная, штука, сейчас, я, смотрю, на...",мир интересная штука смотрю открыто большим д...
251,13299,NaN,"На мой взгляд, на данный момент тема мира очен...","{'text': 'На мой взгляд, на данный момент тема...","[на, мой, взгляд, на, данный, момент, тема, ми...",взгляд данный момент тема мира очень актуальн...
252,13272,NaN,"«Я, другие, мир». Что можно сказать по этой те...","{'text': '«Я, другие, мир». Что можно сказать ...","[я, другие, мир, что, можно, сказать, по, этой...",другие мир сказать теме пытаюсь позиционирова...


In [25]:
# Создание текстовых файлов для обучения fasttext модели с лейблом и текстом
# файлы с минимально обработаннымм текстами
make_txt(train, 'text', 'train-text.txt')
make_txt(test, 'text', 'test-text.txt')

print('Данные для обучения:')
! head -n 10 test-text.txt

Данные для обучения:
__label__nan В плане отношений с другими люди делятся на два типа: экстравертов и интровертов. Наверное, абсолютно каждый человек когда-либо задумывался, а к какому типу людей из этих двух относится он? Естественно, что и я не являюсь исключением, однако задаваясь этим вопросом в моем сознании также встают и другие вопросы. Один из них и, наверно, самый главный: а кого, действительно, можно считать экстравертом? Того, кто всегда открыто разговаривает с теми, кто к нему подойдет и проявит инициативу, или же того, кто сам эту инициативу и проявляет? Однозначно сказать очень тяжело, поэтому я разделяю эти два типа и отношу себя к первому типу экстравертов . Говоря же о себе и отношениях с другими людьми, я задумываюсь над тем, что у меня есть свой собственный маленький мир, в котором и находятся мои близкие, однако, учитывая то, что я отношу себя, к экстравертам первого типа, я предполагаю, что мой мир может расширяться, конечно, не всегда, когда я сам того пожелаю, н

In [31]:
# Вычисление предсказаний для всех текстов теста
# Модель с базовыми параметрами на минимально обработанных данных train
model = fasttext.train_supervised('train-text.txt')

pred = []
for line in test.text:
  pr = model.predict(line)[0][0]
  pred.append(re.sub('__label__',' ', pr))

test.label = pred

#pred
test[['ID', 'label', 'text']].to_csv('test_fasttext.csv', encoding='utf-8-sig')
test[['ID', 'label', 'text']]

,ID,label,text
0,19163,Non-depression,В плане отношений с другими люди делятся на дв...
1,7028,Non-depression,"Я в этом мире как муравей в муравейнике, как л..."
2,5070,Non-depression,"Современные люди стали жестоки,к нашему миру и..."
3,17580,Non-depression,Люди ко мне относится довольна странно я обычн...
4,17445,Non-depression,"Ну так как мне 18 лет, то я еще не совсем пони..."
...,...,...,...
249,6928,Non-depression,"Мир-слово, имеющее несколько значений. Оно упо..."
250,3540,Non-depression,Мир – интересная штука. Сейчас я смотрю на нег...
251,13299,Non-depression,"На мой взгляд, на данный момент тема мира очен..."
252,13272,Non-depression,"«Я, другие, мир». Что можно сказать по этой те..."


Они все Non-depression

#### Теперь для той же задачи использую предобученную модель

In [27]:
predict_dep(test.text, 0.3)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


dep_score =  0.3
0
0.11764705882352941
Non-depression
1
0.02857142857142857
Non-depression
2
0.26666666666666666
Non-depression
3
0.09090909090909091
Non-depression
4
0.06896551724137931
Non-depression
5
0.02564102564102564
Non-depression
6
0.02127659574468085
Non-depression
7
0.125
Non-depression
8
0.0
Non-depression
9
0.26666666666666666
Non-depression
10
0.0
Non-depression
11
0.1
Non-depression
12
0.0
Non-depression
13
0.06451612903225806
Non-depression
14
0.038461538461538464
Non-depression
15
0.36363636363636365
Depression
16
0.05128205128205128
Non-depression
17
0.0
Non-depression
18
0.0
Non-depression
19
0.14285714285714285
Non-depression
20
0.0
Non-depression
21
0.0
Non-depression
22
0.0
Non-depression
23
0.0
Non-depression
24
0.14285714285714285
Non-depression
25
0.0
Non-depression
26
0.05
Non-depression
27
0.0
Non-depression
28
0.0
Non-depression
29
0.037037037037037035
Non-depression
30
0.02564102564102564
Non-depression
31
0.06666666666666667
Non-depression
32
0.05555555555

RuntimeError: ignored

Увеличить размер pipeline не удалось. Но из уже обработанного видно, что вновь выходят сплошлые Non-depression. 